In [13]:
import pandas as pd
from sklearn.metrics import *
from tqdm import tqdm
from utils import HeteroDataProcessorFilterNodeonTestV2
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [14]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"

time_cut =12

processor = HeteroDataProcessorFilterNodeonTestV2(file_path_replies, file_path_posts, time_cut_replies=7, time_cut_posts=12)
data = processor.process()


In [3]:
data

HeteroData(
  id={
    x=[1406, 106],
    y=[1406],
    train_mask=[1406],
    val_mask=[1406],
    test_mask=[1406],
  },
  reply_user_id={ x=[13156, 104] },
  (id, retweet, reply_user_id)={ edge_index=[2, 13156] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 13156] }
)

In [15]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("GAT Network 2024-10-06 2 layers Filter Node on test 17min-3days two time filters")

<Experiment: artifact_location='/home/azureuser/rumour-detection-pheme/mlruns/38', creation_time=1728216302899, experiment_id='38', last_update_time=1728216302899, lifecycle_stage='active', name=('GAT Network 2024-10-06 2 layers Filter Node on test 17min-3days two time '
 'filters'), tags={}>

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero

class GAT(torch.nn.Module):
    def __init__(self, dim_h, dim_out):
        super().__init__()
        self.conv1 = GATConv((-1, -1), dim_h, add_self_loops=False)
        self.conv2 = GATConv(dim_h, dim_h, add_self_loops=False)  # Added second GATConv layer
        self.linear = nn.Linear(dim_h, dim_out)
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index).relu()
        h = self.dropout(h)
        h = self.conv2(h, edge_index).relu()  # Pass through the second GATConv layer
        h = self.dropout(h)
        h = self.linear(h)
        return h

@torch.no_grad()
def test(mask):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
    acc = (pred[mask] == data['id'].y[mask]).sum() / mask.sum()
    return float(acc)

In [19]:

model = GAT(dim_h=64, dim_out=2)
model = to_hetero(model, data.metadata(), aggr='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)



for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)['id']
    mask = data['id'].train_mask
    loss = F.cross_entropy(out[mask], data['id'].y[mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0:
        train_acc = test(data['id'].train_mask)
        val_acc = test(data['id'].val_mask)
        print(f'Epoch: {epoch:>3} | Train Loss: {loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}%')
    
test_acc = test(data['id'].test_mask)
print(f'Test accuracy: {test_acc*100:.2f}%')
    


Epoch:   0 | Train Loss: 0.6383 | Train Acc: 77.66% | Val Acc: 0.00%
Epoch:  50 | Train Loss: 0.3730 | Train Acc: 83.65% | Val Acc: 33.33%
Epoch: 100 | Train Loss: 0.3048 | Train Acc: 86.51% | Val Acc: 66.67%
Epoch: 150 | Train Loss: 0.2559 | Train Acc: 89.15% | Val Acc: 66.67%
Epoch: 200 | Train Loss: 0.2070 | Train Acc: 91.65% | Val Acc: 66.67%
Epoch: 250 | Train Loss: 0.1738 | Train Acc: 94.29% | Val Acc: 66.67%
Epoch: 300 | Train Loss: 0.1356 | Train Acc: 95.72% | Val Acc: 66.67%
Epoch: 350 | Train Loss: 0.0933 | Train Acc: 97.07% | Val Acc: 66.67%
Epoch: 400 | Train Loss: 0.0778 | Train Acc: 97.79% | Val Acc: 66.67%
Epoch: 450 | Train Loss: 0.0652 | Train Acc: 98.79% | Val Acc: 66.67%
Test accuracy: 100.00%


In [20]:
test_mask = data['id'].test_mask | data['id'].val_mask
pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
true_labels = data['id'].y[test_mask]
pred_labels = pred[test_mask]
precision_score(true_labels, pred_labels, average='macro')

0.8333333333333333

In [21]:
recall_score(true_labels, pred_labels, average='macro')

0.8333333333333333

In [ ]:
processor = HeteroDataProcessorFilterNodeonTestV2(file_path_replies, file_path_posts, time_cut_replies=7, time_cut_posts=12)

In [19]:
a = 13
if a==13 then 42 else 80

SyntaxError: invalid syntax (641925305.py, line 2)

In [17]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"

for time_cut_posts in range(132,(60*24*3),60):
    for time_cut_replies in range(7,(60*24*2),30):
        print(time_cut_posts,time_cut_replies)
        processor = HeteroDataProcessorFilterNodeonTestV2(file_path_replies, file_path_posts, time_cut_replies,time_cut_posts)
        data = processor.process()
    
        model = GAT(dim_h=64, dim_out=2)
        model = to_hetero(model, data.metadata(), aggr='sum')
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data, model = data.to(device), model.to(device)
       
        with mlflow.start_run():
    
            for epoch in range(400):
                model.train()
                optimizer.zero_grad()
                out = model(data.x_dict, data.edge_index_dict)['id']
                mask = data['id'].train_mask
                loss = F.cross_entropy(out[mask], data['id'].y[mask])
                loss.backward()
                optimizer.step()
                
                if epoch % 100 == 0:
                    print(epoch)
                    train_acc = test(data['id'].train_mask)
                    val_acc = test(data['id'].val_mask)
                    print(f'Epoch: {epoch:>3} | Train Loss: {loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}%')
    
                
            test_acc = test(data['id'].test_mask)
            print(f'Test accuracy: {test_acc*100:.2f}%')
    
            test_mask = data['id'].test_mask | data['id'].val_mask
            pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
            true_labels = data['id'].y[test_mask]
            pred_labels = pred[test_mask]
            test_precision =precision_score(true_labels, pred_labels, average='macro')
            test_recall=recall_score(true_labels, pred_labels, average='macro')
            print(f'Test Recall: {test_recall*100:.2f}%')
            print(f'Test Precision: {test_precision*100:.2f}%')
    
            mlflow.log_metric("train_loss", loss.item(), step=epoch)
            mlflow.log_metric("train_acc", train_acc, step=epoch)
            mlflow.log_metric("val_acc", val_acc, step=epoch)
            mlflow.log_metric("test_acc", test_acc)
            mlflow.log_metric("test_precision",  test_precision)
            mlflow.log_metric("test_recall",  test_recall)
    
                
            mlflow.log_param("dim_h", 64)
            mlflow.log_param("dim_out", 2)
            mlflow.log_param("learning_rate", 0.001)
            mlflow.log_param("epochs", 400)
            mlflow.log_metric("time_cut_replies", time_cut_replies)
            mlflow.log_metric("time_cut_posts", time_cut_posts)
    
    
    
            
                
            mlflow.pytorch.log_model(model, f"GAT_model_{time_cut}")


132 367
0
Epoch:   0 | Train Loss: 0.6831 | Train Acc: 77.66% | Val Acc: 34.69%
100
Epoch: 100 | Train Loss: 0.2974 | Train Acc: 86.22% | Val Acc: 69.39%
200
Epoch: 200 | Train Loss: 0.1975 | Train Acc: 91.01% | Val Acc: 69.39%
300
Epoch: 300 | Train Loss: 0.1151 | Train Acc: 95.79% | Val Acc: 71.43%
Test accuracy: 89.80%
Test Recall: 72.02%
Test Precision: 76.96%


2024/10/06 14:01:47 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


132 397
0
Epoch:   0 | Train Loss: 0.7611 | Train Acc: 42.54% | Val Acc: 57.14%
100
Epoch: 100 | Train Loss: 0.3054 | Train Acc: 86.65% | Val Acc: 67.35%
200
Epoch: 200 | Train Loss: 0.2193 | Train Acc: 90.51% | Val Acc: 83.67%
300
Epoch: 300 | Train Loss: 0.1413 | Train Acc: 93.36% | Val Acc: 83.67%
Test accuracy: 91.84%
Test Recall: 83.71%
Test Precision: 85.58%


2024/10/06 14:02:07 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


132 427
0
Epoch:   0 | Train Loss: 0.7430 | Train Acc: 74.80% | Val Acc: 34.69%
100
Epoch: 100 | Train Loss: 0.3242 | Train Acc: 84.94% | Val Acc: 63.27%
200
Epoch: 200 | Train Loss: 0.2335 | Train Acc: 89.22% | Val Acc: 69.39%
300
Epoch: 300 | Train Loss: 0.1651 | Train Acc: 94.65% | Val Acc: 75.51%
Test accuracy: 93.88%
Test Recall: 79.02%
Test Precision: 82.75%


2024/10/06 14:02:29 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


132 457
0
Epoch:   0 | Train Loss: 1.0562 | Train Acc: 25.98% | Val Acc: 63.27%
100
Epoch: 100 | Train Loss: 0.3233 | Train Acc: 85.22% | Val Acc: 61.22%


KeyboardInterrupt: 

In [8]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# Get the experiment ID or name
experiment_name = "GAT Network 2024-10-06 2 layers Filter Node on test 17min-3days two time filters"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is not None:
    experiment_id = experiment.experiment_id

    # Retrieve all runs in the experiment as a DataFrame
    df = mlflow.search_runs(experiment_ids=[experiment_id])
    
    # Show the DataFrame
    #print(df)
else:
    print(f"Experiment '{experiment_name}' not found.")


In [11]:
df['metrics.time_cut_posts'].max()

132.0

In [12]:
df[df['metrics.time_cut_posts']==132]['metrics.time_cut_replies'].max()

337.0